In [1]:
import numpy as np
import scipy

# Numerische Quadratur
Definition der Quadratur (Approximation eines Integrals), wobei c die Stützstellen sind und w die Gewichte.
$$
\int_{a}^{b} f(x) \, dx \approx Q_n(f; a, b) :=  \sum_{i=1}^{n} w_i f(c_i).
$$
Definition des Fehlers einer Quadratur
$$
E(n) := \left| \int_{a}^{b} f(x) \, dx - Q_n(f; a, b) \right|
$$
Definition der algebraischen Konvergenz
$E(n) = O\left( \frac{1}{n^p} \right)$

Definition der exponentiellen Konvergenz
$E(n) = O\left( q^n \right)$ mit $0 \leq q < 1$


## 5.4 Äquidistante Stützstellen

### Einfache Regeln zum Approximieren
### Theorem zur Konvergenz
$$
\left| \int_{I} f(t) \, dt - Q(f) \right| \leq C h^p \, \|f^{(p)} \|_{L^\infty(I)}
$$
Der Integrationsfehler hängt von der $p$-ten Ableitung der Funktion $f$ ab.
Die Schranke für den Fehler ist nur anwendbar, falls $f \in C^p(I)$ gilt, also $f$ $p$-fach stetig differenzierbar ist. Ist $f$ hinreichend differenzierbar ($f \in C^p$), so erzielt das Verfahren seine optimale Genauigkeit.
Fehlt jedoch diese Differenzierbarkeitsstufe ($f \notin C^p$), bleibt die erwartete Höchstleistung unerreicht und die Konvergenzgeschwindigkeit liegt unter $p$.

Mittelpunkt-Regel
$$
Q^{M}(f; a, b) = (b-a) f \left( \frac{a+b}{2} \right)
$$


In [1]:
def midpoint(f, a, b, N):
    """
    Midpoint quadrature rule for numerical integration.
    
    Parameters:
    f: Function to integrate, f(x)
    a: Lower bound of integration interval
    b: Upper bound of integration interval
    N: Number of subintervals
    
    Returns:
    I: Approximation of the integral using the midpoint rule
    """
    # Create N+1 equally spaced points from a to b and get step size h
    x, h = np.linspace(a, b, int(N) + 1, retstep=True)
    # Calculate midpoints between consecutive points and evaluate f at these midpoints
    # Sum all contributions: h * f(midpoint) for each subinterval
    I = np.sum(h * f((x[:-1] + x[1:]) / 2))
    return I


Trapez-Regel
$$
Q^{T}(f; a, b) = \frac{b-a}{2} (f(a) + f(b))
$$


In [ ]:
def trapezoidal(f, a, b, N):
    """
    Trapezoidal quadrature of function f from a to b with N subintervals.
    f: Function f(x)
    a, b: Bounds of the integration interval
    N: Number of subintervals
    """
    x, h = np.linspace(a, b, int(N)+1, retstep=True)
    # quadrature weights:
    # boundary nodes: w=1/2
    # internal nodes: w=1
    I = h/2.0 * (f(x[0]) + 2.0*sum(f(x[1:-1])) + f(x[-1]))
    return I

Simpson-Regel
$$
Q^{S}(f; a, b) = \frac{b-a}{6} \left ( f(a) + 4 f\left (\frac{a+b}{2} \right) + f(b) \right)
$$

In [ ]:
def simpson(f, a, b, N):
    """
    Simpson quadrature of function f from a to b with N subintervals.
    f: Function f(x)
    a, b: Bounds of the integration interval
    N: Number of subintervals
    this method uses 2*N+1 function evaluations
    """
    x, h = np.linspace(a, b, 2*int(N)+1, retstep=True)
    # quadrature weights:
    # boundary nodes: w=1/3
    # internal nodes: w=2/3
    # midpoint nodes: w=4/3
    I = h/3.0 * sum(f(x[:-2:2]) + 4.0*f(x[1:-1:2]) + f(x[2::2]))
    return I

In [ ]:
# !TODO Romberg Schema

## 5.5 Nicht äquidistante Stützstellen
### Definition Orthogonale Polynome:
Für eine Gewichtsfunktion $\omega(x)$ ($w\ne\omega$) definieren wir das Skalarprodukt wie folgt.
$$
\langle f, g \rangle := \int_{a}^{b} f(x) g(x) \, \omega(x) \, dx
$$
Theorem: Für ein wie oben definiertes Skalarprodukt existiert eine eindeutige Folge
von Polynomen $p0$, $p1$, $p2$, . . . sodass diese paarweise orthogonal sind und $deg(pi) = i$
gilt.
#### Definition des Skalarprodukts für Legendre-Polynome
Die Gewichts funktion ist $\omega(x) = 1$
$$
\langle f, g \rangle := \int_{-1}^{1} f(x) g(x) \, dx
$$

### Jacobi Polynome
Die Legendre-Polynome sind ein Spezialfall der Jacobi-Polynome.
Jacobi-Polynome sind orthogonal bezüglich des Skalarprodukts:
$$
\langle f, g \rangle := \int_{-1}^{1} f(x) g(x) (1-x)^\alpha (1+x)^\beta \, dx
$$
Durch geeignete Wahl von $\alpha$ und $\beta$ bekommt man neue Quadraturen.
####
### 5.5.1 Gaus Quadratur
**Motivation**:
Die Gauss-Quadratur macht keine Annahme über Abstände (also nicht äquidistant),
sondern sucht Punkte und Gewichte so, dass Polynome bis zum einem möglichst hohen
Grad exakt integriert werden

**Definition**
$$
\int_{-1}^{1} f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)
$$

#### Gaus-Legendre Quadratur
Die Gewichts funktion ist $\omega(x) = 1$.

$\alpha = 0, \beta = 0$

**Theorem**: Die Knoten $x_1, . . . , x_n$, die diese Eigenschaft erfüllen, sind die Nullstellen
des Legendre-Polynoms $p_n(x)$.

In [ ]:
def gauss_legendre(f, a, b, n):
    r"""Integrate f, over [a, b], using Gauss-Legendre quadrature.

    f       : function to integrate
    a       : left end
    b       : right end
    n       : degree of the quadrature rule
    """
    xn, wn = scipy.special.roots_legendre(n)
    x = 0.5 * (b-a) * xn + (a+b) / 2
    w = 0.5 * (b-a) * wn
    return np.sum(w*f(x))


#### Gaus-Radau Quadratur
Die Gewichtsfunktion ist $omega(x) = 1$.
Falls der rechte Rand fixiert ist, gilt: $\alpha = 1, \beta = 0$, ansonsten falls der linke Rand fixiert ist: $\alpha = 0, \beta = 1$.

In [1]:
def gauss_radau(f, a, b, n, fixed='r'):
    """
    Integrate f over [a, b] using Gauss–Radau quadrature.

    Parameters
    ----------
    f : callable
        Function to integrate.
    a, b : float
        Integration interval.
    n : int
        Number of nodes (including the fixed one).
    fixed : {'r', 'l'}, optional
        Fixed endpoint:
        - 'r': right endpoint (x = 1) included.
        - 'l': left endpoint (x = -1) included.
    """
    if fixed == 'r':
        # Right endpoint fixed at x=1
        r, w = scipy.special.roots_jacobi(n-1, alpha=1, beta=0)
        nodes = np.hstack((r, [1.0]))
        weights = np.hstack((w / (1 - r), [2 / n**2]))
    else:
        # Left endpoint fixed at x=-1
        r, w = scipy.special.roots_jacobi(n-1, alpha=0, beta=1)
        nodes = np.hstack(([-1.0], r))
        weights = np.hstack(([2 / n**2], w / (1 + r)))

    # Map from [-1, 1] → [a, b]
    x = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    w = 0.5 * (b - a) * weights

    return np.sum(w * f(x))

#### Gaus-Lobatto Quadratur
Die Gewichtsfunktion ist $omega(x) = 1$.
Beide Ränder der Knoten sind fixiert. $\alpha = 1, \beta = 1$

In [2]:
def gauss_lobatto(f, a, b, n):
    """
    Integrate f over [a, b] using Gauss–Lobatto quadrature
    with n nodes (including endpoints), using roots_jacobi.

    Parameters
    ----------
    f : callable
        Function to integrate.
    a, b : float
        Integration interval.
    n : int
        Number of nodes (including the fixed one).
    """
    if n < 2:
        raise ValueError("Gauss–Lobatto requires at least 2 nodes")

    if n == 2:
        # Only endpoints
        nodes = np.array([-1.0, 1.0])
        weights = np.array([1.0, 1.0])
    else:
        # Internal nodes and weights from Jacobi(n-2,1,1)
        internal_nodes, internal_weights = scipy.special.roots_jacobi(n-2, alpha=1, beta=1)

        # All nodes
        nodes = np.hstack(([-1.0], internal_nodes, [1.0]))

        # All weights
        weights = np.zeros(n)
        weights[0] = 2 / (n * (n-1))      # left endpoint
        weights[-1] = 2 / (n * (n-1))     # right endpoint

        # Internal weights: scale Jacobi weights
        weights[1:-1] = internal_weights * 2 / (n*(n-1))

    # Map from [-1,1] → [a,b]
    x = 0.5*(b - a)*nodes + 0.5*(a + b)
    w = 0.5*(b - a)*weights

    return np.sum(w * f(x))

# 5.7 Quadratur in $\mathbb{R}^d$ und dünne Gitter?
Zwar sind Dünne Gitter nicht Prüfungsrelevant, so ist Quadratur in $\mathbb{R}^d$ generell nicht unrelevant

Inhalt:
- Integrale von Funktionen $f:\mathbb{R}^d -> \mathbb{R}$
- Naiver Ansatz durch Tensor Produkt
- Sparse Grids aus spass?

**Später** wird die Quadratur in $\mathbb{R}^d$ im Kapitel [5.8 Monte Carlo Methoden]( 
weitergeführt
Generell gilt analytisch durch Fubinis Theorem:  
$$ X \in \mathbb{R}^d$$
$$\int_{\Omega}f(X)d(X) = \int_{a_1}^{b_1}\int_{a_2}^{b_2}\dots\int_{a_d}^{b_d}f(x,...,x_d)dx_d...dx_2dx_1$$

Illustration Anhand vom Fall $\mathbb{R}^2$  
$$\int_a^b\int_a^bf(x,y)dxdy = \int_a^bF(y)dy$$

$$I = \sum_{j_2=1}^{n_1}\omega_{j_1}w_{j_2}f(c_{j_1},c_{j_2})$$

Zunächst **Implementation von diesem Naiven Approach**  
Problem:
wenn wir einer der Regeln Trapez, Mittelpunkt oder Simpson anwenden wollen
müssen wir zunächst die Gewichte finden.

**Summierte-Trapez-Regel**
Aufspalten in Gewichte und Knoten:  
$$I^T(f;a,b) \approx \dfrac{h}{2}f(a) + \dfrac{h}{2}2*\sum_{i=1}^{N-1}f(x_i) + \dfrac{h}{2}f(b)$$
$$=>$$
$$\omega_i = h, i \in \{1,...,N-1\}, \omega_0 = \omega_N = \dfrac{h}{2}$$
$$c_i = np.linspace(a,b,N+1)$$
Meshgrid:
$$
\begin{array}{c|ccccc}
  & x_0 & x_1 & x_2 & x_3 & x_4 \\
\hline
y_0 & f(x_0,y_0) & f(x_1,y_0) & f(x_2,y_0) & f(x_3,y_0) & f(x_4,y_0) \\
y_1 & f(x_0,y_1) & f(x_1,y_1) & f(x_2,y_1) & f(x_3,y_1) & f(x_4,y_1) \\
y_2 & f(x_0,y_2) & f(x_1,y_2) & f(x_2,y_2) & f(x_3,y_2) & f(x_4,y_2) \\
y_3 & f(x_0,y_3) & f(x_1,y_3) & f(x_2,y_3) & f(x_3,y_3) & f(x_4,y_3) \\
y_4 & f(x_0,y_4) & f(x_1,y_4) & f(x_2,y_4) & f(x_3,y_4) & f(x_4,y_4)
\end{array}
$$
Erklärung:
Aus der verschachtelten Summe sehen wir, dass wir jedes Knotenpaar benötigen für die Auswertung. Meshgrid = Kartesischen Produkt(x,y)
$$
\begin{array}{c|ccccc}
  & x_0 & x_1 & x_2 & x_3 & x_4 \\
\hline
y_0 & (x_0,y_0) & (x_1,y_0) & (x_2,y_0) & (x_3,y_0) & (x_4,y_0) \\
y_1 & (x_0,y_1) & (x_1,y_1) & (x_2,y_1) & (x_3,y_1) & (x_4,y_1) \\
y_2 & (x_0,y_2) & (x_1,y_2) & (x_2,y_2) & (x_3,y_2) & (x_4,y_2) \\
y_3 & (x_0,y_3) & (x_1,y_3) & (x_2,y_3) & (x_3,y_3) & (x_4,y_3) \\
y_4 & (x_0,y_4) & (x_1,y_4) & (x_2,y_4) & (x_3,y_4) & (x_4,y_4)
\end{array}
$$
Gewichte Multipliziert:
$$
\begin{array}{c|ccccc}
  & x_0 & x_1 & x_2 & x_3 & x_4 \\
\hline
y_0 & \tfrac{h^2}{4}f(x_0,y_0) & \tfrac{h^2}{2}f(x_1,y_0) & \tfrac{h^2}{2}f(x_2,y_0) & \tfrac{h^2}{2}f(x_3,y_0) & \tfrac{h^2}{4}f(x_4,y_0) \\
y_1 & \tfrac{h^2}{2}f(x_0,y_1) & h^2 f(x_1,y_1) & h^2 f(x_2,y_1) & h^2 f(x_3,y_1) & \tfrac{h^2}{2}f(x_4,y_1) \\
y_2 & \tfrac{h^2}{2}f(x_0,y_2) & h^2 f(x_1,y_2) & h^2 f(x_2,y_2) & h^2 f(x_3,y_2) & \tfrac{h^2}{2}f(x_4,y_2) \\
y_3 & \tfrac{h^2}{2}f(x_0,y_3) & h^2 f(x_1,y_3) & h^2 f(x_2,y_3) & h^2 f(x_3,y_3) & \tfrac{h^2}{2}f(x_4,y_3) \\
y_4 & \tfrac{h^2}{4}f(x_0,y_4) & \tfrac{h^2}{2}f(x_1,y_4) & \tfrac{h^2}{2}f(x_2,y_4) & \tfrac{h^2}{2}f(x_3,y_4) & \tfrac{h^2}{4}f(x_4,y_4)
\end{array}
$$
Erklärung:
Wie oben Illustriert sind die Gewichte der Randpunkte bei der Trapezregel unteschiedlich. Beim meshgrid haben wir die Randpunkte von x und y auf dem Rand.
Ergebnis: Summe von Elementen im Mesh
**Annahme**: Hier nehmen wir an, dass $(b-a)/N_x = (d-c)/N_y$, also dass auf der x und y achse die Intervalle Identisch sind. Falls anders, so hätten wir verschiedene Intervallängen h also $h_x = (b-a)/N_x$, $h_y = (d-c)/N_y$

**Summierte-Simpson-Regel**:
$$I^S(f;a,b) \approx \dfrac{h}{6}(f(a) + 2\sum_{i=1}^{N-1}f(x_i) + 4\sum_{i=1}^Nf(\dfrac{x_{i-1} + x_i}{2}) + f(b))$$
$$\omega_0 = \omega_{2N} = \dfrac{h}{6}$$
$$\omega_{2i} = \dfrac{h}{6}*2, i \in \{i\,even\,and\,not\,0\,or\,2N\}$$
$$\omega_{2i} = \dfrac{h}{6}*4, i \in\{i\,odd\,and\,not\,0\,or\,2N\}$$
$$c_i = np.linspace(a,b,2N+1)$$
Die anwendung ist hier ähnlich. Man muss einfach beachten, wo die Gewichte sich Wiederholen im Meshgrid. 
Dem Leser ist es überlassen, wie man das fuer Simpson implementiert indem die Trapezregel in $\mathbb{R}^2$ verstanden wird.

Mögliche Umsetzung in $\mathbb{R}^d$ für ein Viereck (sollte sehr ähnlich sein in $\mathbb{R}^2$. Für **Gauss Quadratur** sollte es jetzt relativ intuitiv sein wie man es implementiert


In [ ]:
#Code-Expert-Week6 Task 4
import numpy as np
def trapezoid2d(f, a, b, Nx, c, d, Ny):
    """Approximiert das Doppelintegral mit der Trapezregel.

    Input:
           f : Funktion f(x, y) welche integiert werden soll.
        a, b : untere/obere Grenze des Integrals nach 'dx'.
          Nx : Anzahl Teilintervalle des Integrals nach 'dx'.
        c, d : untere/obere Grenze des Integrals nach 'dy'.
          Ny : Anzahl Teilintervalle des Integrals nach 'dy'.
    """
    h1 = (b-a)/(Nx) # Intervallänge X-Achse 
    h2 = (d-c)/(Ny) # Intervallänge Y-Achse
    c1 = np.linspace(a,b,Nx+1)
    c2 = np.linspace(c,d,Ny+1)
    cartesian_product = np.array(np.meshgrid(c1,c2),dtype=float)
    function_vals = f(cartesian_product[0],cartesian_product[1])
    length = len(function_vals[0])
    height = len(function_vals[:,0])
    function_vals[[0, -1], :] /= 2.0
    function_vals[:, [0, -1]] /= 2.0
    return np.sum(h1*h2*function_vals)  # Bitte ersetzen.
def simpson2d(f, a, b, Nx, c, d, Ny):
    """Approximiert das Doppelintegral mit der Trapezregel.

    Input:
           f : Funktion f(x, y) welche integiert werden soll.

        a, b : untere, obere Grenze des Integrals nach 'dx'.
          Nx : Anzahl Teilintervalle für das Integral über 'x'.

        c, d : untere, obere Grenze des Integrals nach 'dy'.
          Ny : Anzahl Teilintervalle für das Integral über 'y'.
    """
    h1 = (b-a)/Nx
    h2 = (d-c)/Ny
    c1 = np.linspace(a,b,2*Nx+1)
    c2 = np.linspace(c,d,2*Ny+1)
    point_mesh = np.meshgrid(c1,c2)
    f_vals = f(point_mesh[0], point_mesh[1])
    f_vals[1::2, :] *= 4/6.0
    f_vals[2:-1:2, :] *= 2/6.0
    f_vals[:, 1::2] *= 4/6.0
    f_vals[:, 2:-1:2] *= 2/6.0
    f_vals[[0, -1], :] /= 6.0
    f_vals[:, [0, -1]] /= 6.0
    # TODO vervollständigen Sie das Template.

    return np.sum(h1*h2*f_vals) # Bitte ersetzen.
